```{margin} Adaptation!
This lecture was adapted with permission from Prof. AJ Medford (GTech)'s lectures for ChBE4745: https://github.com/medford-group/data_analytics_ChE

The dataset came from Dow Chemicals and released publicly as part of Prof. Medford's class.
```

`````{note}
This lecture is going to:
* introduce the functional form of neural networks
* code a neural network from scratch using numpy/scipy
* show why gradients are so important for fitting models with many parameters
* improve the efficiency of regression using automatic differentiation
* discuss some of the logistical considerations of using and training neural networks
    * mini-batch
    * gradient descent
    * packages, accelerators, etc
`````

## Unsupervised learning: dimensionality reduction

Let's remind ourselves what supervised and unsupervised means in the context of machine learning:
* **supervised:** We are building models that have some input data/features and some known output target/label (a number if regression, a categorical variable if classification)
* **unsupervised:** We want models that only use the input data/features.

Since we're often trying to predict something in engineering we've focused on supervised techniques so far. However, unsupervised learning is very helpful. The two most common use cases are:
* **dimensionality reduction:** We want to reduce the number of input features
* **clustering:** We want to cluster some data to find similar points in a dataset or understand the data distribution

We're going to talk about dimensionality reduction today!

## Dataset: Dow process impurity

We're going to use the Dow process dataset that you saw on your homework already. 


![DOW process](resources/dow_process.png)

The dataset contains a number of operating conditions for each of the units in the process, as well as the concentration of impurities in the output stream. We'll use the same filtering that you used on your homework to remove some problematic data points. 

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_excel("datasets/impurity_dataset-training.xlsx")


def is_real_and_finite(x):
    if not np.isreal(x):
        return False
    elif not np.isfinite(x):
        return False
    else:
        return True


all_data = df[df.columns[1:]].values  # drop the first column (date)
numeric_map = df[df.columns[1:]].applymap(is_real_and_finite)
real_rows = (
    numeric_map.all(axis=1).copy().values
)  # True if all values in a row are real numbers
X = np.array(
    all_data[real_rows, :-5], dtype="float"
)  # drop the last 5 cols that are not inputs
y = np.array(all_data[real_rows, -3], dtype="float")

x_names = [str(x) for x in df.columns[1:41]]
y_name = str(df.columns[-3])

Let's remind ourselves how big this dataset is (# of points and # of features)

In [ ]:
print(f"There are {X.shape[0]} data points in the impurity dataset")
print(f"There are {X.shape[1]} features in the impurity dataset")

Let's also make a simple 80/20 train/test split. This is important since some of the data analysis techniques will be used to build better supervised models.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=42
)

One of the challenges with data with 40 dimensions is that it's extremely hard to visualize. 2-3 dimensions is pretty straightforward, but 40 is impossible!

## Visualization of features

Unlike working with a single variable where we can plot "x vs. y", but it is difficult to get a feel for higher-dimension data since it is hard to visualize. One good thing to start with is looking at histograms of each input variable. This is super easy using dataframes!

In [ ]:
df[df.columns[1:41]].hist(bins=30, figsize=(30, 20));

We can see that some features are normally distributed, while others have some obvious outliers or bimodal distribution. We have no idea how these features are correlated yet - that is if any of them are related.

## Feature correlations

There are 40 features in the dataset, but from our engineering knowledge we expect that some might end up being correlated. For example, if there's an energy balance, the energy in one unit may be directly correlated with the energy in another unit or stream. 

We can formalize this with a **correlation coefficient**. The most common/useful correlation coefficient is the Pearson correlation coefficient. It is a number in the range [-1,1] that describes how correlated two variables are. I find this plot from the wikipedia page to be extremely helpful!

![correlation examples from wikipedia article](resources/Correlation_examples2.svg)

`````{seealso}
https://en.wikipedia.org/wiki/Pearson_correlation_coefficient
`````

Let's try this for our data! `df.corr()` calculates the correlation coefficient for all columns in the dataset

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Found this snippet by googling "correlation matrix plotly"
# and finding https://stackoverflow.com/questions/66572672/correlation-heatmap-in-plotly

pio.templates.default = "plotly_white"

fig = go.Figure()
fig.add_heatmap(
    z=df[df.columns[1:41]].corr(),
    x=x_names,
    y=x_names,
    colorscale=px.colors.diverging.RdBu,
    zmin=-1,
    zmax=1,
)
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

This plot has some very interesting structure!
* The diagonal is 1 - every feature is perfectly correlated with itself
* The plot is symmetric - the correlation between x1 and x2 is the same as the correlation between x2 and x1
* Many pairs have a strong positive (~1) correlation
* Some pairs have a very weak correlation
* A few pairs have strong negative correlation
* There are some obvious groups among the features. For example, all of the primary column bed temperatures are strongly correlated with each other.


Let's plot a few of these just to see what happens. 

### Example of positive correlation

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

fig = go.Figure()
fig.add_scatter(
    x=df["x18:Primary Column Bed 4 Temperature"],
    y=df["x19:Primary Column Bed 3 Temperature"],
    mode="markers",
)
fig.update_layout(autosize=False, width=600, height=600)
fig.update_xaxes(title_text="x18:Primary Column Bed 4 Temperature")
fig.update_yaxes(title_text="x19:Primary Column Bed 3 Temperature")
fig.show()

It's probably not so surprising that the temperature in beds next to each other in the same column are pretty strongly correlated! 

Interestingly there are a few strong outliers here - that could either be noise or erroneous datapoints, or could be really interesting and rare scenarios. We don't really know unless we dig into the actual data. I would probably select a few of those conditions and investigate what happened at those specific times!

### Example of negative correlation
Let's try one of the negative ones from the matrix above

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

fig = go.Figure()
fig.add_scatter(
    x=df["x34: Secondary Column Tails Temperature"],
    y=df["x22: Secondary Column Base Concentration"],
    mode="markers",
)
fig.update_layout(autosize=False, width=600, height=600)
fig.update_xaxes(title_text="x34: Secondary Column Tails Temperature")
fig.update_yaxes(title_text="x22: Secondary Column Base Concentration")
fig.show()

This one is a little less clear - there's clearly a correlated. All of the points with high tails temperature in the second column also have a very low base concentration. I can't explain this without thinking a little more about the chemical engineering process, but it immediately jumps out from the data. 

Be careful though - correlation is not causation!

## Dimensionality reduction

We can take advantage of the fact that many of the features are correlated to reduce the number of features in our system. From the example above, we probably don't need all of the temperatures in the beds of the first column, unless those outliers happen to be important!

Many dimensionality reduction techniques are implemented in scikit-learn. We'll try just two simple ones here.

### Practical uses of dimensionality reduction

There are a number of practical uses for dimensionality reduction algorithms:

* compression of data
* denoising of data
* interpretation of data
* improving model efficiency or performance

We will focus primarily on the ways that dimensionality reduction can aid in interpretation and improving model efficiency and performance, but the algorithms used for other applications are the same or similar.

### Considerations for dimensionality reduction

There are many different kinds of dimensionality reduction approaches, and when selecting between them there are a few things to consider. The relative importance of these factors will depend on the nature of the dataset and the goal of the analysis.

* Matrix rank - how many independent dimensions are there?
* Linearity of the low-dimensional subspace - are patterns linear or non-linear?
* Projection - can a new high-dimensional point be projected onto the low-dimensional map?
* Inversion - can a new low-dimensional point be projected back into high-dimensional space?
* Supervised vs. unsupervised - are the training labels used to determine the reduced dimensions?

### Assessing performance of dimensionality reduction models

It can be challenging to assess the performance of dimensional reduction models, especially when unsupervised. Nonetheless there are a few approaches that can be used. Selecting the right approach will depend on the problem, but using a variety of assessment criteria is always a good idea if possible.

#### Explained Variance (most common) 
One common idea in dimensional reduction is to assess the "explained variance" of the high-dimensional data. This is common in techniques such as PCA.

#### Distance

The "stress" function compares the distance between points $i$ and $j$ in a low-dimensional space to the distance in the full-dimensional space:

$ S(\vec{x}_{0}, \vec{x}_1, \vec{x}_2, ... \vec{x}_n) =  \left( \frac{\sum_{i=0}^n \sum_{i<j}(d_{ij} - ||x_i - x_j||)^2}{\sum_{i=0}^n \sum_{i<j} d_{ij}^2} \right)^{1/2} $


where $d_{ij}$ is the distance in the high-dimensional space and $\vec{x}$ is the vector in the low-dimensional space.

A conceptually similar way to express this is:

$\sum_i \sum_j || d(\vec{x}_i, \vec{x}_j) - d(P(\vec{x}_i), P(\vec{x}_j))||$

where $d(\vec{x}_i, \vec{x}_j)$ is the distance between $\vec{x}_i$ and $\vec{x}_j$ in the high-dimensional space, and $P(\vec{x}_j)$ is the reduced-dimension vector.

Some approaches seek to minimize these distances directly (e.g. multi-dimensional scaling), but it can also be used as an accuracy metric. We can implement this using a few helper functions. You don't need to worry about the details of this function, but can look up the documentation to see the connection.

### Principal component analysis

You can identify linear combinations of the original features that contain independent information using principal component analysis (PCA). PCA works by using the eigenvectors of the covariance matrix to identify linear combinations. 
 The eigenvectors of the covariance matrix identify the "natural" coordinate system of the data.

![correlation examples from wikipedia article](resources/PCA.gif)



PCA isn't too hard to implement from scratch, but we'll use the sklearn interface since the details are not so important. Unsupervised methods like PCA have a similar interface to sklearn ML models, but instead of a `predict` function they have a `transform` function.

`````{seealso}
sklearn PCA: https://scikit-learn.org/stable/modules/decomposition.html#pca

scatter matrix: https://plotly.com/python/splom/
`````

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA

# Default PCA object
pca = PCA()

# Fit the PCA and transform the data
components = pca.fit_transform(X_train)

# Get the explained variance from the PCA object
# and format it as a string
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

# Plot with plotly!
fig = px.scatter_matrix(
    components,
    labels=labels,
    hover_data={
        a: b for a, b in zip(x_names, X_train.T)
    },  # add the rest of the data on hover!
    dimensions=range(4),
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

Notice that the first three principal components explain almost all of the data as most of the features are correlated. We could use these three components as features in a supervised ML model. You will try that in your homework!

### t-SNE manifold learning

t-SNE (t-distributed stochastic neighbor embedding) is another popular method for learning low-dimensional representation of high-dimensional data. t-SNE operates by trying to find a low-dimensional representation that minimizes the stress above. Effectively, you want to learn a manifold such that points that are close in the new space are also close in the high-dimensional space. It is particularly helpful for clustering high-dimensional data.

As the name implies, tSNE is stochastic, which means that the results you get will change each time you run it unless you  set the seed.

Let's see how it does! This code takes a while to run (~2min).

`````{seealso}
tSNE in sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

Original tSNE paper (2008): https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf
`````

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Default PCA object
tsne = TSNE(init="pca", n_iter=1000, verbose=2)

# Fit the PCA and transform the data
components = tsne.fit_transform(X_train)

In [ ]:
# Plot with plotly!
fig = px.scatter(
    x=components[:, 0],
    y=components[:, 1],
    hover_data={
        a: b for a, b in zip(x_names, X_train.T)
    },  # add the rest of the data on hover!
    color=y_train,
)
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

We can see that there are some significant clusters that have emerged, and if you zoom in the larger clusters also have clusters. Probably this is coming from multiple data points collected near each other in time. 